# Part 2 - Modelling
## Chapter 6 - Cross Validation in Finance

### 7.1 Why is shuffling a dataset before conducting k-fold CV generally a bad idea in finance? What is the purpose of shuffling? Why does shuffling defeat the purpose of k-fold CV in financial datasets?

Shuffling is bad idea because of data leakage and non IID samples, if we shuffle the data we cant monitor if there is a leakage from our training to the testing model. makes the k fold irrelevant because each test set has similar samples in training set and it may lead to false features discoveries.  

### 7.2 Take a pair of matrices (X, y), representing observed features and labels. These could be one of the datasets derived from the exercises in Chapter 3.
#### 7.2 (a) Derive the performance from a 10-fold CV of an RF classifier on (X, y), without shuffling.
#### 7.2 (b) Derive the performance from a 10-fold CV of an RF on (X, y), with shuffling.
#### 7.2 (c) Why are both results so different?
#### 7.2 (d) How does shuffling leak information?